# Import Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp # Go through Media pipe

# Keypoints using MP Holistic

In [3]:
mp_holistic =mp.solutions.holistic # Bringing holsitic model
mp_drawing =mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image,model):
    image =cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # COLOR CONVERSION - BGR 2 RGB
    image.flags.writeable= False                 # Image is no longer writeable (Helps in saving memory)
    results=model.process(image)                 # Make Predictions
    image.flags.writeable=True                   # Image is now writeable
    image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # COLOR CONVERSION - RGB 2 BGR
    return image,results

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [6]:
# Does the same thing as the draw_landmarks function except helps in customizing the landmarks
def draw_styled_landmarks(imag,results):
    # Draw Face Connections
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)                            
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(256,256,121),thickness=2,circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             )
# mp_drawing.draw_landmarks??

In [6]:
cap=cv2.VideoCapture(0)
#Setting mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :  
    while cap.isOpened():
        # Reading the feed
        ret,frame = cap.read()
        # frame = cv2.flip(frame,1)

        # Making the detection
        image,results = mediapipe_detection(frame,holistic)
    
        # Draw Landmarks
        # draw_landmarks(image,results)
        draw_styled_landmarks(image,results)

        # Show to screen
        cv2.imshow('OpenCV Feed',image)

        # Breaking gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
results.right_hand_landmarks

# Extract Keypoint Values

The variable pose is equivalent to this results.pose_landmarks.landmark

`print('Length of both pose and result.pose_landmarks.landmarks is equal to each other = {}, {}'.format(len(results.pose_landmarks.landmark),len(pose)))`

In [8]:
pose=[]
for res in results.pose_landmarks.landmark:
    test=np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)
# print(len(pose)) # Just to check with with value in pose_without_flatten in the next code
# pose

**# Use this block only to see the output** 

*# Using list comprehension to make it smaller*

`pose_without_flatten=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark])`

`print(pose_without_flatten.shape)`

`pose_without_flatten`

In [7]:
def extract_keypoints(results):    
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    # Similarly for left hand and right hand
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)  # The if statement is for when there is no reading for yout left hand
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  # The if statement is for when there is no reading for yout right hand
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,face,lh,rh]) # RETURNS ALL THE KEYPOINTS IN ONE 2D First 132 values is pose keypoints next is 468 value value of face keypoint and so on

In [10]:
# Next three lines of code is just for explanation
result_test=extract_keypoints(results)
np.save('0',result_test) 

In [11]:
result_test

array([ 0.48526436,  0.56312144, -1.09927189, ...,  0.        ,
        0.        ,  0.        ])

In [12]:
np.load('0.npy')

array([ 0.48526436,  0.56312144, -1.09927189, ...,  0.        ,
        0.        ,  0.        ])

# Setup Folders for Collection

In [8]:
# Path for exported data,numpy arrays
DATA_PATH = os.path.join('MP_DATA')

# Actions that we try to detect
actions = np.array(['Hello','Thank You'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# Collecting Data from Key Points

In [42]:
cap=cv2.VideoCapture(0)
#Setting mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :

    # NEW LOOP
    #Loop through our actions  
    for action in actions:
        
        # Loop thoruh sequences aka videos
        for sequence in range(no_sequences):

            #Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # Reading the feed
                ret,frame = cap.read()
                # frame = cv2.flip(frame,1)

                # Making the detection
                image,results = mediapipe_detection(frame,holistic)
            
                # Draw Landmarks
                # draw_landmarks(image,results)
                draw_styled_landmarks(image,results)

                # NEW Apply wait Logic
                if frame_num == 0:
                    cv2.putText(image,'Starting Collection',(120,200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255), 1,cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed',image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12),
                                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255), 1,cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed',image)
                # NEW Export Keypoints
                keypoints=extract_keypoints(results)
                npy_path =os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)

            # Breaking gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [10]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [17]:
label_map={label:num for num,label in enumerate(actions)}
label_map

{'Hello': 0, 'Thank You': 1}

In [44]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [45]:
x=np.array(sequences)
x.shape # Total Videos , 30 videos per action , 1662 keypoints 

(60, 30, 1662)

In [46]:
y=to_categorical(labels).astype(int) # Gives each action a value of 1 in the respective column so in this example for the first 30 rows the first column will have 1 indicating 'Hello' and in the next 30 rows column 2 will have 1 indicating 'I love you'
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)

In [47]:
y_test.shape

(6, 2)

# Build and Train LSTM Neural Network

In [52]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [53]:
log_dir =os.path.join('Logs')
tb_callback =TensorBoard(log_dir=log_dir)

In [54]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu',))
model.add(LSTM(64,return_sequences=False,activation='relu',))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [12]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [56]:
model.fit(x_train,y_train,epochs=500,callbacks=[tb_callback])

Epoch 1/500
2/2 [==============================] - 3s 105ms/step - loss: 1.3791 - categorical_accuracy: 0.4815
Epoch 2/500
2/2 [==============================] - 0s 98ms/step - loss: 14.3693 - categorical_accuracy: 0.4815
Epoch 3/500
2/2 [==============================] - 0s 102ms/step - loss: 45.5915 - categorical_accuracy: 0.5370
Epoch 4/500
2/2 [==============================] - 0s 104ms/step - loss: 14.3756 - categorical_accuracy: 0.3889
Epoch 5/500
2/2 [==============================] - 0s 98ms/step - loss: 18.5894 - categorical_accuracy: 0.5185
Epoch 6/500
2/2 [==============================] - 0s 103ms/step - loss: 13.5430 - categorical_accuracy: 0.5185
Epoch 7/500
2/2 [==============================] - 0s 100ms/step - loss: 11.4167 - categorical_accuracy: 0.5000
Epoch 8/500
2/2 [==============================] - 0s 102ms/step - loss: 11.5727 - categorical_accuracy: 0.6111
Epoch 9/500
2/2 [==============================] - 0s 104ms/step - loss: 9.3067 - categorical_accuracy: 0.5

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 2)                 66        
                                                                 
Total params: 596,642
Trainable params: 596,642
Non-tr

# Make Predictions

In [60]:
x_test

array([[[ 0.54890811,  0.5431788 , -1.03008461, ...,  0.52551174,
          0.91088772, -0.10750023],
        [ 0.50920773,  0.55131429, -1.03795671, ...,  0.50667828,
          0.74109137, -0.07778063],
        [ 0.4921802 ,  0.56244045, -1.19387794, ...,  0.4982346 ,
          0.7351917 , -0.0809406 ],
        ...,
        [ 0.46946993,  0.53312629, -1.24612176, ...,  0.49510545,
          0.69650799, -0.08131244],
        [ 0.47940359,  0.53133285, -1.26690984, ...,  0.49684271,
          0.67320639, -0.07802243],
        [ 0.48175588,  0.52727407, -1.26672554, ...,  0.49851361,
          0.65839934, -0.07093037]],

       [[ 0.49838385,  0.63344067, -1.17017674, ...,  0.14355257,
          0.37566465, -0.02160176],
        [ 0.51108348,  0.63571858, -1.01321197, ...,  0.32521674,
          0.49017349, -0.02115152],
        [ 0.51509881,  0.6359629 , -1.05747628, ...,  0.32607567,
          0.48707902, -0.02364166],
        ...,
        [ 0.51850843,  0.6417284 , -1.0582093 , ...,  

In [61]:
res= model.predict(x_test)
print(res.shape)
actions[np.argmax(res[0])]

1/1 [==============================] - 0s 34ms/step
(6, 2)


'Thank You'

In [58]:
actions[np.argmax(y_test[0])]

'Thank You'

# Save Weights

In [62]:
model.save('action.h5')

In [51]:
del model

In [11]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu',))
model.add(LSTM(64,return_sequences=False,activation='relu',))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))
model.load_weights('action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [11]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [64]:
yhat=model.predict(x_train)

ytrue=np.argmax(y_train,axis=1).tolist()

yhat=np.argmax(yhat,axis=1).tolist()

2/2 [==============================] - 0s 24ms/step


In [65]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[28,  0],
        [ 0, 26]],

       [[26,  0],
        [ 0, 28]]], dtype=int64)

In [79]:
accuracy_score(ytrue,yhat)

1.0

# Testing in Real Time

In [13]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [14]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.98
predictions=[]

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        cv2.waitKey(20)
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [15]:
cap.release()
cv2.destroyAllWindows()

In [77]:
np.argmax(res)

7